In [24]:
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import os

#사용자 입력 받기
print("=" *80)
print("     영화 '명량' 리뷰 추출하기")
print("=" *80)
cnt = int(input('    1.해당 분야에서 크롤링 할 건수는 몇건입니까?(1-100 건 사이 입력): '))

query_txt = "영화'명량'리뷰 추출하기"
f_dir = input("    2.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
print("\n")

#드라이버를 이용하여 영화 "명량"에 접속
path = "C:\py_temp\chromedriver.exe"
driver = webdriver.Chrome(path)
url = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=93756"
driver.get(url)


#저장 폴더 및 파일이름 정하기
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-')
os.chdir(f_dir+s+'-'+query_txt+'-')

ff_dir=f_dir+s+'-'+query_txt+'-'
ff_name=f_dir+s+'-'+query_txt+'-'+'\\'+s+'-'+query_txt+'-'+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+'\\'+s+'-'+query_txt+'-'+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+'\\'+s+'-'+query_txt+'-'+'.xls'

s_time = time.time( )

# 홈페이지 html 받아오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#평점 더보기 버튼 누르기
driver.find_element_by_xpath('''//*[@id="content"]/div[1]/div[4]/div[5]/div[2]/a''').click()
time.sleep(3)

# 관람객평점에서 별점, 리뷰내용, 작성자, 작성일자, 공감횟수, 비공감횟수 가져오기
writer_id2=[]
review2=[]
write_date2=[]
gogam=[]
gogam_0=[]
gogam_1=[]
score2=[]

f = open(ff_name, 'a',encoding='UTF-8')
    
count = 0
page_count = 1

while(cnt>=count):
    
    driver.switch_to_default_content()
    driver.switch_to_frame('pointAfterListIframe')

    slist = soup.find('div', class_="score_result").find('ul').find_all('li')    
    
    for li in slist:
            count += 1
            print("\n")
            print("%s 번째 댓글 수집 중 ==================" %count)

            writer_id = li.find('dt').find('span').get_text()
            print("1.작성자ID:", writer_id)
            f.write("\n")
            f.write("%s 번째 리뷰 데이터를 수집합니다==============" %count + "\n")
            f.write("1.작성자ID:"+writer_id + "\n")
            writer_id2.append(writer_id)

            try :
              review = li.find('p').get_text()
            except AttributeError :
                review='작성자에 의해 삭제된 댓글입니다'
                print("2.리뷰 :",review)
            else :
                print("2.리뷰:",review)
            f.write("2.리뷰:" + review + "\n")
            review2.append(review)

            write_date = li.find('dt').find_all('em')
            write_date = write_date[1].get_text()
            print('3.작성일자:',write_date)
            f.write("3.작성일자:" + write_date + "\n")
            write_date2.append(write_date)

            gogam = li.find('a', class_='_sympathyButton').find_all('strong')

            try :
              g_gogam = gogam[0].text
              print('4.공감:',g_gogam)
            except IndexError :
              g_gogam = '0'
              print('4.공감 :',g_gogam)
            f.write("4.공감:" + g_gogam + "\n")
            gogam_0.append(g_gogam)

            gogam = li.find('a', class_='_notSympathyButton').find_all('strong')

            try :
                b_gogam = gogam[0].text
                print('5.비공감:',b_gogam) 
            except IndexError :
              b_gogam = '0'
              print('5.비공감 :',b_gogam)
            f.write("5.비공감:" + b_gogam + "\n")
            gogam_1.append(b_gogam)

            score = li.find('div', class_='star_score').find('em').get_text()
            print("6.별점:", score)
            f.write("6.별점:"+score + "\n")
            score2.append(score)

            print("\n")        
            time.sleep(0.2)
            if(cnt == count):
                break;
    
    if(cnt == count):
            break;            
    driver.find_element_by_class_name('pg_next').click()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

print("%s 건  완료========================================================" %count)
time.sleep(random.randrange(3,8))  # 3-8 초 사이에 랜덤으로 시간 선택
                                       
#xls형태와 csv 형태로 저장하기
movie_reple = pd.DataFrame()
movie_reple['작성자ID']=pd.Series(writer_id2)
movie_reple['리뷰내용']=pd.Series(review2)
movie_reple['작성일자']=pd.Series(write_date2)
movie_reple['공감횟수']=pd.Series(gogam_0)
movie_reple['비공감횟수']=pd.Series(gogam_1)
movie_reple['별점']=pd.Series(score2)

# csv 형태로 저장하기
movie_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
movie_reple.to_excel(fx_name,index=True)

driver.close( )

     영화 '명량' 리뷰 추출하기
    1.해당 분야에서 크롤링 할 건수는 몇건입니까?(1-100 건 사이 입력): 21
    2.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):C:\py_temp\naver\




c:\python35\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: use driver.switch_to.default_content instead
c:\python35\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: use driver.switch_to.frame instead




1 번째 댓글 수집 중 ==================
1.작성자ID: chocobar(choc****)
2.리뷰: 

														
														
														
														
															
															
																오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네 맛없네 까대며 입터는것뿐. 한심한 것들. 
															
														
														
													
3.작성일자: 2014.07.30 01:31
4.공감: 16784
5.비공감: 4983
6.별점: 10




2 번째 댓글 수집 중 ==================
1.작성자ID: 누구세요(h100****)
2.리뷰: 

														
														
														
														
															
															
																이순신, 그는 신이다. 
															
														
														
													
3.작성일자: 2014.07.30 00:02
4.공감: 12899
5.비공감: 2264
6.별점: 10




3 번째 댓글 수집 중 ==================
1.작성자ID: 드리머(hama****)
2.리뷰: 

														
														
														
														
															
															
																군도보다 훨씬 잼있음..극장가서 꼭 보세요 
															
														
														
													